In [2]:
import requests
from urllib.parse import quote
import pandas as pd
import numpy as np
import time
import json

In [3]:
API_KEY = "RGAPI-73f368a8-5abe-41e9-8297-b022e477d21c"
GAME_NAME = "X8Scorpio8X"
TAG_LINE = "NA1"  # do not include "#"
REGION = "Americas"

def get_puuid_from_riot_id(game_name, tag_line, api_key, region):
    # URL-encode the game name and tag line
    game_name_encoded = quote(game_name)
    tag_line_encoded = quote(tag_line)
    
    url = f"https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name_encoded}/{tag_line_encoded}"
    headers = {"X-Riot-Token": api_key}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data["puuid"]
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Example usage
puuid = get_puuid_from_riot_id(GAME_NAME, TAG_LINE, API_KEY, REGION)
if puuid:
    print("PUUID:", puuid)


PUUID: vrfSU4Nh3GYoxHi2hLXg_yOczDqdE8WulZzWDLl2RZu6wtmH-e8i629HTEVbu1Uej1CpJZ3JIfXu3w


In [43]:
PUUID = puuid
REGION = "americas"  # match routing region

def get_match_ids(puuid, api_key, region, count=100):
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    params = {
        "start": 0,
        "count": count  # number of matches to retrieve (max 100 per call)
    }
    headers = {"X-Riot-Token": api_key}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()  # list of match IDs
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Example usage
match_ids = get_match_ids(PUUID, API_KEY, REGION)
if match_ids:
    print("Recent Match IDs:", match_ids)


Recent Match IDs: ['NA1_5344630817', 'NA1_5344601433', 'NA1_5344568732', 'NA1_5344546140', 'NA1_5344490992', 'NA1_5344450710', 'NA1_5344393544', 'NA1_5344369275', 'NA1_5344329639', 'NA1_5343839050', 'NA1_5343819906', 'NA1_5343763274', 'NA1_5343745959', 'NA1_5343707171', 'NA1_5343679593', 'NA1_5343613957', 'NA1_5343573278', 'NA1_5343529584', 'NA1_5343501569', 'NA1_5342992849', 'NA1_5342944672', 'NA1_5342906501', 'NA1_5342868647', 'NA1_5342831384', 'NA1_5341881351', 'NA1_5341122227', 'NA1_5341099101', 'NA1_5341076721', 'NA1_5340015356', 'NA1_5339979888', 'NA1_5339938860', 'NA1_5339898889', 'NA1_5339848963', 'NA1_5339805279', 'NA1_5339764387', 'NA1_5339717890', 'NA1_5339525454', 'NA1_5339184544', 'NA1_5339145751', 'NA1_5339107184', 'NA1_5339061539', 'NA1_5339030682', 'NA1_5338975460', 'NA1_5338074472', 'NA1_5338006815', 'NA1_5337968788', 'NA1_5336550559', 'NA1_5336510101', 'NA1_5336160869', 'NA1_5336143295', 'NA1_5336128941', 'NA1_5335276972', 'NA1_5334930751', 'NA1_5334909429', 'NA1_5334

In [48]:
def parse_match_data(match_data):
    if match_data['info']['queueId'] != 420 and match_data['info']['queueId'] != 400:  # Only solo/duo ranked games or draft pick normal games
        return []

    teams = match_data['info']['teams']
    participants = match_data['info']['participants']

    # Split participants by teamId
    team1_id = teams[0]['teamId']
    team2_id = teams[1]['teamId']
    team1_players = [p for p in participants if p['teamId'] == team1_id]
    team2_players = [p for p in participants if p['teamId'] == team2_id]

    def team_stats(team, players, other_players):
        def get_ping_sum(key):
            return sum(p.get(key, 0) for p in players)

        game_duration_minutes = match_data['info']['gameDuration'] / 60

        # Calculate damage per minute
        magic_dpm = sum(p['magicDamageDealtToChampions'] for p in players) / game_duration_minutes
        physical_dpm = sum(p['physicalDamageDealtToChampions'] for p in players) / game_duration_minutes
        true_dpm = sum(p['trueDamageDealtToChampions'] for p in players) / game_duration_minutes

        # Calculate damageCV (coefficient of variation)
        damage_values = [magic_dpm, physical_dpm, true_dpm]
        mean_damage = sum(damage_values) / len(damage_values)
        std_damage = (sum((x - mean_damage) ** 2 for x in damage_values) / len(damage_values)) ** 0.5 if mean_damage != 0 else 0
        damage_cv = std_damage / mean_damage if mean_damage != 0 else 0

        total_pings = (
            get_ping_sum('allInPings') +
            get_ping_sum('assistMePings') +
            get_ping_sum('commandPings') +
            get_ping_sum('enemyMissingPings') +
            get_ping_sum('enemyVisionPings') +
            get_ping_sum('holdPings') +
            get_ping_sum('getBackPings')
        )

        return {
            'matchId': match_data['metadata']['matchId'],
            'teamId': team['teamId'],
            'win': team['win'],
            'baronKills': team['objectives']['baron']['kills'],
            'dragonKills': team['objectives']['dragon']['kills'],
            'riftHeraldKills': team['objectives']['riftHerald']['kills'],
            'totalGold': sum(p['goldEarned'] for p in players),
            'goldSpent': sum(p['goldSpent'] for p in players),
            'averageChampionExp': sum(p['champExperience'] for p in players) / len(players),
            'wardsPlaced': sum(p['wardsPlaced'] for p in players),
            'allInPings': get_ping_sum('allInPings'),
            'assistMePings': get_ping_sum('assistMePings'),
            'commandPings': get_ping_sum('commandPings'),
            'enemyMissingPings': get_ping_sum('enemyMissingPings'),
            'enemyVisionPings': get_ping_sum('enemyVisionPings'),
            'holdPings': get_ping_sum('holdPings'),
            'getBackPings': get_ping_sum('getBackPings'),
            'totalPings': total_pings,
            'magicDPM': magic_dpm,
            'physicalDPM': physical_dpm,
            'trueDPM': true_dpm,
            'damageCV': damage_cv
        }

    return [
        team_stats(teams[0], team1_players, team2_players),
        team_stats(teams[1], team2_players, team1_players)
    ]


def collect_match_stats(match_ids, api_key):
    all_rows = []
    for i, match_id in enumerate(match_ids):
        url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
        headers = {"X-Riot-Token": api_key}
        
        while True:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                match_data = response.json()
                rows = parse_match_data(match_data)
                all_rows.extend(rows)
                break
            elif response.status_code == 429:
                print("Rate limit exceeded. Sleeping for 60 seconds...")
                time.sleep(60)
            else:
                print(f"Failed to retrieve match {match_id}: {response.status_code}")
                break

        time.sleep(1.3)

    return pd.DataFrame(all_rows)



In [51]:
#TESTER SESSION

names_taglines = {"X8Scorpio8X":"NA1"}
puuids = []
region = "americas"

# getting puuids of silver players
for GAME_NAME in names_taglines:
    puuids.append(get_puuid_from_riot_id(GAME_NAME, names_taglines[GAME_NAME], API_KEY, region))

# getting match_ids from players' recent games
match_ids = []
for puuid in puuids:
    match_ids.append(get_match_ids(puuid, API_KEY, region))

match_ids = np.array(match_ids).flatten().tolist()



# collecting match stats
scorpioData = collect_match_stats(match_ids, API_KEY)

# creating Coefficient of variation for each team's damage
scorpioData['damageCV'] = scorpioData[['magicDPM', 'physicalDPM', 'trueDPM']].std(axis=1) / scorpioData[['magicDPM', 'physicalDPM', 'trueDPM']].mean(axis=1)

In [52]:
scorpioData.to_csv("X8Scorpio8X_Data")
scorpioData.size

3916

Collecting data from people who are in silver rank

In [42]:
# each set of names_taglines represents a wave
names_taglines0 = {"Valdivian":"NA1", "Gray Goo":"NA1", "Taljedyn":"2304", "Zalsdunt":"2644", "AdaptedMahoraga":"4375"}
names_taglines1 = {"Bunnie":"9356", "Nizannden":"NA1", "TouchGrass":"2EZ", "LonelyGod":"Osaka", "loosylegumes9":"LL9", "Fordyddoc":"NA1"}

names_taglines2 = {"Paro":"0820", "MicDonuld":"NA1", "prohanp":"NA1", "eGirl Trainer": "queer", "iSejuani":"NA1", "Fartgas12":"NA1"}

names_taglines3 = {"7hunder3":"Lucky", "grinder enjoyer":"nocap", "Atomic":"8541", "LucQie":"NA1", "Konlen":"NA1", "Hide Inside Lux":"NA1"}

In [49]:
# run this cell in waves for each set of names_taglines so that the code actually runs
names_taglines = names_taglines3
puuids = []
region = "americas"

# getting puuids of silver players
for GAME_NAME in names_taglines:
    puuids.append(get_puuid_from_riot_id(GAME_NAME, names_taglines[GAME_NAME], API_KEY, region))

# getting match_ids from players' recent games
match_ids = []
for puuid in puuids:
    match_ids.append(get_match_ids(puuid, API_KEY, region, count=15))

match_ids = np.array(match_ids).flatten().tolist()

In [50]:
# collecting match stats
silver_data = collect_match_stats(match_ids, API_KEY)

#creating Coefficient of variation for each team's damage
silver_data['damageCV'] = silver_data[['magicDPM', 'physicalDPM', 'trueDPM']].std(axis=1) / silver_data[['magicDPM', 'physicalDPM', 'trueDPM']].mean(axis=1)

In [51]:
silver_data.to_csv('silver_data.csv', mode='a', header=False)
silver_data

,matchId,teamId,win,baronKills,dragonKills,riftHeraldKills,totalGold,goldSpent,totalKills,totalDeaths,...,mid,midItems,bot,botItems,sup,supItems,magicDPM,physicalDPM,trueDPM,damageCV
0,NA1_5324219691,100,True,1,3,1,72233,63900,35,32,...,Yone,"[1055, 3032, 3006, 3031, 6673, 6333, 3340]",Xayah,"[3036, 3508, 6675, 1037, 3172, 1053, 3363]",Senna,"[3009, 3071, 1018, 3877, 1038, 1037, 3364]",875.953533,2553.949661,137.628267,1.041256
1,NA1_5324219691,200,False,0,2,0,65632,62650,32,35,...,Ahri,"[3157, 1082, 6653, 3020, 1058, 3118, 3340]",Ashe,"[3006, 3085, 3031, 3072, 6670, 3032, 3340]",Karma,"[3871, 3158, 3118, 3916, 2503, 4005, 3364]",2655.333979,1295.866409,372.110358,0.796972
2,NA1_5323683182,100,False,0,1,1,59621,57425,25,38,...,Xerath,"[1058, 6655, 1082, 3020, 4645, 1058, 3340]",Jinx,"[3032, 3085, 3172, 3031, 3036, 1036, 3340]",Ashe,"[3871, 3006, 3179, 3087, 3044, 1037, 3364]",2103.575130,1283.005181,99.450777,0.867054
3,NA1_5323683182,200,True,1,4,0,69041,62383,38,25,...,Malzahar,"[3040, 6653, 3020, 3802, 3116, 2055, 3340]",Tristana,"[3032, 3006, 3031, 1038, 6675, 3036, 3363]",Shen,"[3876, 3084, 2502, 3076, 1036, 3047, 3364]",1381.056995,1927.585492,182.238342,0.767220
4,NA1_5323657628,100,True,1,2,1,58277,50458,32,7,...,Ziggs,"[4629, 6653, 2055, 3040, 3041, 3020, 3340]",Lucian,"[3508, 3036, 1038, 1055, 2422, 1037, 3340]",Velkoz,"[3871, 3020, 3916, 2503, 2508, 1052, 3364]",1607.214886,2693.445378,280.120048,0.791567
5,NA1_5323657628,200,False,0,2,0,43336,40316,7,32,...,Ahri,"[6657, 3158, 6653, 3108, 0, 0, 3340]",Ezreal,"[0, 3042, 3078, 2021, 0, 3158, 3340]",Braum,"[3876, 1001, 3190, 3109, 0, 0, 3364]",1450.372149,1184.585834,142.905162,0.746299
6,NA1_5323636257,100,True,1,3,0,58318,51200,34,21,...,Neeko,"[2420, 3152, 4646, 1056, 3175, 0, 3340]",Ashe,"[1055, 3032, 3006, 3046, 3072, 0, 3340]",Nautilus,"[3869, 3173, 3075, 3065, 1028, 0, 3364]",1527.392097,2004.765957,131.708207,0.796965
7,NA1_5323636257,200,False,0,1,1,49058,43800,21,34,...,Kassadin,"[1054, 3158, 2031, 6657, 3802, 3070, 3340]",Sivir,"[3032, 3031, 3006, 6675, 1036, 1036, 3340]",Maokai,"[3876, 3009, 3002, 2502, 0, 0, 3364]",942.237082,813.957447,77.617021,0.763307
8,NA1_5322954806,100,True,1,3,0,71212,65663,39,22,...,Akali,"[1082, 4646, 3175, 3089, 4645, 6653, 3340]",Jinx,"[3085, 3172, 6672, 3036, 1053, 1038, 3340]",Velkoz,"[3871, 3175, 6655, 6653, 2503, 0, 3364]",2519.800285,2660.199715,446.305278,0.660994
9,NA1_5322954806,200,False,0,2,1,59711,57168,21,39,...,Akshan,"[3087, 3031, 6676, 1036, 6670, 3013, 3340]",Ezreal,"[1055, 3158, 3078, 3042, 6694, 2021, 3340]",Bard,"[3877, 3742, 6653, 3114, 3009, 3067, 3364]",1648.930100,1711.554922,134.778887,0.766314


In [52]:
silver_data["totalInhibitorsTaken"]

0     1
1     2
2     0
3     2
4     2
5     0
6     2
7     0
8     1
9     0
10    0
11    0
12    2
13    0
14    0
15    0
16    2
17    0
18    0
19    2
20    0
21    0
22    0
23    0
24    1
25    0
26    0
27    3
28    0
29    5
30    2
31    1
32    1
33    3
34    2
35    0
36    0
37    2
38    0
39    2
40    0
41    0
42    0
43    1
44    1
45    0
46    0
47    2
48    0
49    0
50    0
51    1
52    0
53    1
54    5
55    0
56    3
57    1
58    3
59    0
Name: totalInhibitorsTaken, dtype: int64